In [4]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [5]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data
import xarray as xr
import h5py
import pickle

# Helpful
import time
import datetime
import itertools
from itertools import product
from tqdm import tqdm
import os

# My Methods
import importlib
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
from src.models.EMOS_global.EMOS_global_load_models import *

<IPython.core.display.Javascript object>

### Goal: Denormalize data

In [6]:
var_names = ["u10", "v10", "t2m", "t850", "z500"]

<IPython.core.display.Javascript object>

#### 1. Load Data

In [18]:
dat_train_proc_norm = ldp.load_data_all_train_proc_norm()
dat_test_proc_norm = ldp.load_data_all_test_proc_norm()

<IPython.core.display.Javascript object>

In [19]:
X_train_var_lead_all, y_train_var_lead_all = split_var_lead(dat_train_proc_norm)
X_test_lead_all, y_test_var_lead_all = split_var_lead(dat_test_proc_norm)

<IPython.core.display.Javascript object>

#### 2. Load Means and Stds

In [24]:
X_train_var_lead_all[4][0]

<xarray.DataArray 'z500_train' (forecast_date: 1429, lat: 120, lon: 130,
                                mean_std: 2)>
array([[[[-1.001446,  0.037566],
         ...,
         [-0.999621,  0.03127 ]],

        ...,

        [[ 0.988863,  0.024431],
         ...,
         [ 0.80914 ,  0.034025]]],


       ...,


       [[[-0.178663,  0.032926],
         ...,
         [-0.687408,  0.027959]],

        ...,

        [[ 0.54886 ,  0.033244],
         ...,
         [ 0.464479,  0.031441]]]], dtype=float32)
Dimensions without coordinates: forecast_date, lat, lon, mean_std

<IPython.core.display.Javascript object>

1. Variable Index Positions:<br>
    u10: 0 <br>
    v10: 1 <br>
    t2m: 2 <br>
    t850: 5 <br>
    z500: 14 <br>

In [11]:
global_means = np.load("/mnt/sda/Data2/fourcastnet/data/stats_v0/global_means.npy")
global_stds = np.load("/mnt/sda/Data2/fourcastnet/data/stats_v0/global_stds.npy")

<IPython.core.display.Javascript object>

In [12]:
global_means = global_means.flatten()[[0, 1, 2, 5, 14]]

<IPython.core.display.Javascript object>

In [13]:
global_stds = global_stds.flatten()[[0, 1, 2, 5, 14]]

<IPython.core.display.Javascript object>

In [15]:
global_means

array([-6.42138980e-03,  2.93685094e-01,  2.79020667e+02,  2.74948004e+02,
        5.41704141e+04])

<IPython.core.display.Javascript object>

In [16]:
global_stds

array([   5.61282367,    4.77990804,   21.42417831,   15.74041515,
       3439.80138962])

<IPython.core.display.Javascript object>

In [14]:
dat_train_proc_norm[0][list(dat_train_proc_norm[0].data_vars.keys())[0]]

<xarray.DataArray 'u10_train' (forecast_date: 1429, lead_time: 32, lat: 120,
                               lon: 130, mean_std: 2)>
[1426713600 values with dtype=float32]
Dimensions without coordinates: forecast_date, lead_time, lat, lon, mean_std

<IPython.core.display.Javascript object>

In [60]:
def denormalize(mean, std, x):
    denormalized = (x * std) + mean
    return denormalized

<IPython.core.display.Javascript object>

In [61]:
def denormalize_std(std, x):
    denormalized_std = x * std
    return denormalized_std

<IPython.core.display.Javascript object>

In [82]:
def main(mean, std, dataset, name):
    """
    Denormalize a Dataset and save it at data directory
Args:
    mean (float): mean value to denormalize with
    std (float): standard deviation used to denormalize with
    dataset(dataset): xarray dataset with var_train and var_truth dataArrays
    name (String): name to be given to dataset
Returns:
    None
    
    
    """
    denormalized_mean = xr.apply_ufunc(
        denormalize,
        mean,
        std,
        dataset[list(dataset.data_vars.keys())[0]].isel(mean_std=0),
    )
    denormalized_std = xr.apply_ufunc(
        denormalize_std,
        std,
        dataset[list(dataset.data_vars.keys())[0]].isel(mean_std=1),
    )
    denormalized_truth = xr.apply_ufunc(
        denormalize, mean, std, dataset[list(dataset.data_vars.keys())[1]]
    )
    denormalized_train = xr.concat(
        [denormalized_mean, denormalized_std], dim="mean_std"
    )
    denormalized_train = denormalized_train.transpose(
        "forecast_date", "lead_time", "lat", "lon", "mean_std"
    )
    denormalized_dataset = xr.Dataset(
        data_vars={"u10_train": denormalized_train, "u10_truth": denormalized_truth,}
    )
    denormalized_dataset.to_netcdf(
        "/Data/Delong_BA_Data/mean_ens_std_denorm/" + name + ".h5", format="NETCDF4"
    )

<IPython.core.display.Javascript object>

In [63]:
denormalized_mean = xr.apply_ufunc(
    denormalize, -6.42, 5.61, dat_train_proc_norm[0].u10_train.isel(mean_std=0)
)

<IPython.core.display.Javascript object>

In [64]:
denormalized_std = xr.apply_ufunc(
    denormalize_std, -5.61, dat_train_proc_norm[0].u10_train.isel(mean_std=1)
)

<IPython.core.display.Javascript object>

In [65]:
denormalized_truth = xr.apply_ufunc(
    denormalize, -6.42, 5.61, dat_train_proc_norm[0].u10_truth
)

<IPython.core.display.Javascript object>

In [73]:
denormalized_train = xr.concat([denormalized_mean, denormalized_std], dim="mean_std")

<IPython.core.display.Javascript object>

In [74]:
denormalized_train = denormalized_train.transpose(
    "forecast_date", "lead_time", "lat", "lon", "mean_std"
)

<IPython.core.display.Javascript object>

In [27]:
dat_train_proc_norm[0].u10_train.values.shape

(1429, 32, 120, 130, 2)

<IPython.core.display.Javascript object>

In [29]:
test = xr.Dataset(
    data_vars={
        "u10_train": dat_train_proc_norm[0].u10_train,
        "u10_truth": denormalized,
    }
)

<IPython.core.display.Javascript object>

In [36]:
test.to_netcdf("/Data/Delong_BA_Data/mean_ens_std_denorm/test.h5", format="NETCDF4")

<IPython.core.display.Javascript object>

In [37]:
test2 = xr.open_dataset("/Data/Delong_BA_Data/mean_ens_std_denorm/test.h5")

<IPython.core.display.Javascript object>